5. Reinforcement Learning-
a. Calculating Reward b. Discounted Reward
c. Calculating Optimal quantities
d. Implementing Q Learning.
c. Setting up an Optimal Action

In [18]:
import numpy as np
import random
import pandas as pd

# Load the trading dataset
df = pd.read_csv('data/trading.csv')

# Display the first few rows of the dataset
print(df.head())


   Unnamed: 0  datadate   tic      adjcp       open       high        low  \
0           0  20090102  AAPL  12.964286  12.268571  13.005714  12.165714   
1           1  20090102   AXP  19.330000  18.570000  19.520000  18.400000   
2           2  20090102    BA  45.250000  42.800000  45.560000  42.780000   
3           3  20090102   CAT  46.910000  44.910000  46.980000  44.710000   
4           4  20090102  CSCO  16.960000  16.410000  17.000000  16.250000   

       volume  macd    rsi        cci    adx  turbulence  
0  26641980.0   0.0  100.0  66.666667  100.0         0.0  
1  10955620.0   0.0  100.0  66.666667  100.0         0.0  
2   7010171.0   0.0  100.0  66.666667  100.0         0.0  
3   7116726.0   0.0    0.0  66.666667  100.0         0.0  
4  40977480.0   0.0  100.0  66.666667  100.0         0.0  


In [19]:
# Calculate the daily return as reward
df['reward'] = df['adjcp'].pct_change()  
df['reward'] = df['reward'].fillna(0)

In [20]:
# Initialize the 'discounted_reward' column as float
df['discounted_reward'] = 0.0  # Set as float instead of integer

# Calculate discounted rewards
for t in range(len(df)):
    if t == 0:
        df.at[t, 'discounted_reward'] = df.at[t, 'reward']
    else:
        df.at[t, 'discounted_reward'] = df.at[t, 'reward'] + gamma * df.at[t - 1, 'discounted_reward']

# Display the updated DataFrame
print(df[['datadate', 'reward', 'discounted_reward']].head())


   datadate    reward  discounted_reward
0  20090102  0.000000           0.000000
1  20090102  0.491019           0.491019
2  20090102  1.340921           1.782838
3  20090102  0.036685           1.641239
4  20090102 -0.638457           0.838659


In [21]:
# For demonstration, let's define a simple strategy based on rewards
df['optimal_quantity'] = df['reward'].apply(lambda x: 1 if x > 0 else -1)  # Buy (1) if reward is positive, sell (-1) if negative

# Display the DataFrame with optimal quantities
print(df[['datadate', 'reward', 'optimal_quantity']].head())


   datadate    reward  optimal_quantity
0  20090102  0.000000                -1
1  20090102  0.491019                 1
2  20090102  1.340921                 1
3  20090102  0.036685                 1
4  20090102 -0.638457                -1


In [22]:
import numpy as np
import random

# Initialize Q-table
states = range(len(df))  # States could be the index of the DataFrame
actions = [0, 1, -1]  # 0: hold, 1: buy, -1: sell
Q = np.zeros((len(df), len(actions)))

# Hyperparameters
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
gamma = 0.9  # Discount factor

# Q-learning algorithm
for episode in range(100):  # Number of episodes
    state = random.choice(states)  # Start from a random state
    for t in range(len(df) - 1):  # Loop through time steps
        # Choose action (Explore vs Exploit)
        if random.uniform(0, 1) < epsilon:  # Explore
            action_index = random.choice([0, 1, 2])  # Pick a random index for the action (0 -> hold, 1 -> buy, 2 -> sell)
        else:  # Exploit
            action_index = np.argmax(Q[state])  # Choose best action index from the Q-table

        action = actions[action_index]  # Map action_index to actual action value (0, 1, or -1)

        # Perform action and observe the reward
        reward = df.at[t, 'reward'] if action == 1 else -df.at[t, 'reward'] if action == -1 else 0
        
        # Get the next state
        next_state = min(state + 1, len(df) - 1)

        # Update Q-value
        Q[state, action_index] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action_index])

        state = next_state  # Move to the next state

# Display the Q-table
print("Q-Table:\n", Q)


Q-Table:
 [[0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 ...
 [4.96354012 1.167423   1.22216428]
 [5.42494461 1.98873377 1.46851743]
 [4.33767547 5.65171285 4.19859752]]
Optimal action for the last state: Buy


In [23]:
# Determine optimal action for the last state
optimal_action = np.argmax(Q[-1])  # Optimal action for the last state
action_dict = {0: 'Hold', 1: 'Buy', -1: 'Sell'}
print(f"Optimal action for the last state: {action_dict[optimal_action]}")


Optimal action for the last state: Buy
